In [1]:
# 为环境变量做好准备
import os
from io import StringIO
from dotenv import load_dotenv

env_text = """
HOME=./eztransfer_modelzoo
"""

load_dotenv(stream=StringIO(env_text), override=True, verbose=True)
os.environ.get("HOME")

'./eztransfer_modelzoo'

In [2]:
# 让本地的 easytransfer 的优先级更高
import sys
sys.path.insert(0, os.path.abspath('../'))
sys.path

['d:\\code\\github\\EasyTransfer',
 'd:\\code\\github\\EasyTransfer\\examples',
 'c:\\Users\\tzh\\.vscode\\extensions\\ms-toolsai.jupyter-2021.11.1001550889\\pythonFiles',
 'c:\\Users\\tzh\\.vscode\\extensions\\ms-toolsai.jupyter-2021.11.1001550889\\pythonFiles\\lib\\python',
 'C:\\Anaconda3\\envs\\nlp\\python37.zip',
 'C:\\Anaconda3\\envs\\nlp\\DLLs',
 'C:\\Anaconda3\\envs\\nlp\\lib',
 'C:\\Anaconda3\\envs\\nlp',
 '',
 'C:\\Users\\tzh\\AppData\\Roaming\\Python\\Python37\\site-packages',
 'C:\\Anaconda3\\envs\\nlp\\lib\\site-packages',
 'C:\\Anaconda3\\envs\\nlp\\lib\\site-packages\\win32',
 'C:\\Anaconda3\\envs\\nlp\\lib\\site-packages\\win32\\lib',
 'C:\\Anaconda3\\envs\\nlp\\lib\\site-packages\\Pythonwin',
 'C:\\Anaconda3\\envs\\nlp\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\tzh\\.ipython']

In [2]:
# 导入 easytransfer, 并验证, 因为原始的没有 __version__ 这个参数
import importlib
import easytransfer
importlib.reload(easytransfer)
easytransfer.__version__

C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1)

INFO:tensorflow:*********** tf.__version__ is 1.13.2 ******


AttributeError: module 'easytransfer' has no attribute '__version__'

In [2]:
# jupyter 特殊操作
old_argv = sys.argv
print(old_argv)
sys.argv = old_argv[:1]

['C:\\Anaconda3\\envs\\nlp\\lib\\site-packages\\ipykernel_launcher.py', '--ip=127.0.0.1', '--stdin=9003', '--control=9001', '--hb=9000', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"6dfe9eea-9ff8-47f3-a521-a2f335e0efda"', '--shell=9002', '--transport="tcp"', '--iopub=9004', '--f=C:\\Users\\tzh\\AppData\\Local\\Temp\\tmp-45620Vs5j8w6FvsXg.json']


In [3]:
import json

from easytransfer import Config, base_model, layers, model_zoo, preprocessors
from easytransfer.datasets import CSVReader, CSVWriter
from easytransfer.evaluators import classification_eval_metrics
from easytransfer.losses import softmax_cross_entropy

import tensorflow as tf

# 设置随机种子
tf.set_random_seed(32)

C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1)

INFO:tensorflow:*********** tf.__version__ is 1.13.2 ******


In [4]:
class TextClassification(base_model):
    """
    定义文本分类模型
    """

    def __init__(self, **kwargs):
        super(TextClassification, self).__init__(**kwargs)
        self.user_defined_config = kwargs["user_defined_config"]

    def build_logits(self, features, mode=None):
        """构图

        Args:
            features ([type]): [description]
            mode ([type], optional): [description]. Defaults to None.

        Returns:
            [type]: [description]
        """
        # 负责对原始数据进行预处理，生成模型需要的特征，比如：input_ids, input_mask, segment_ids等
        preprocessor = preprocessors.get_preprocessor(
            self.pretrain_model_name_or_path, user_defined_config=self.user_defined_config
        )
        # 负责构建网络的backbone
        model = model_zoo.get_pretrained_model(self.pretrain_model_name_or_path)

        dense = layers.Dense(self.num_labels, kernel_initializer=layers.get_initializer(0.02), name="dense")
        input_ids, input_mask, segment_ids, label_ids = preprocessor(features)
        _, pooled_output = model([input_ids, input_mask, segment_ids], mode=mode)
        logits = dense(pooled_output)

        # 用于 continue finetune
        # self.check_and_init_from_checkpoint(mode)
        return logits, label_ids

    def build_loss(self, logits, labels):
        """定义损失函数

        Args:
            logits ([type]): logits returned from build_logits
            labels ([type]): labels returned from build_logits

        Returns:
            [type]: [description]
        """
        return softmax_cross_entropy(labels, self.num_labels, logits)

    def build_eval_metrics(self, logits, labels):
        """定义评估指标

        Args:
            logits ([type]): logits returned from build_logits
            labels ([type]): labels returned from build_logits

        Returns:
            [type]: [description]
        """
        return classification_eval_metrics(logits, labels, self.num_labels)

    def build_predictions(self, output):
        """定义预测输出

        Args:
            output ([type]): returned from build_logits

        Returns:
            [type]: [description]
        """
        logits, _ = output
        predictions = dict()
        index = tf.argmax(logits, axis=-1, output_type=tf.int32)
        predictions["predict_index"] = index
        predictions["predict_softmax"] = tf.nn.softmax(logits)
        # 核心是理解 shape, 最后一维才是类别数量, 第一个维度是 batch_size
        predictions["predict_prob"] = tf.gather(tf.nn.softmax(logits), index, axis=-1)
        return predictions

In [5]:
def train(config_json):
    config = Config(mode="train_and_evaluate_on_the_fly", config_json=config_json)
    app = TextClassification(user_defined_config=config)

    train_reader = CSVReader(
        input_glob=app.train_input_fp, is_training=True, input_schema=app.input_schema, batch_size=app.train_batch_size,
    )
    eval_reader = CSVReader(
        input_glob=app.eval_input_fp, is_training=False, input_schema=app.input_schema, batch_size=app.eval_batch_size,
    )

    app.run_train_and_evaluate(train_reader=train_reader, eval_reader=eval_reader)


def predict(config_json):
    config = Config(mode="predict_on_the_fly", config_json=config_json)
    app = TextClassification(user_defined_config=config)

    pred_reader = CSVReader(
        input_glob=app.predict_input_fp,
        is_training=False,
        input_schema=app.input_schema,
        batch_size=app.predict_batch_size,
    )
    pred_writer = CSVWriter(output_glob=app.predict_output_fp, output_schema=app.output_schema)

    result = app.run_predict(reader=pred_reader, writer=None, checkpoint_path=app.predict_checkpoint_path)
    for row in result:
        print(row)
        break


def evaluate(config_json):
    config = Config(mode="evaluate_on_the_fly", config_json=config_json)
    app = TextClassification(user_defined_config=config)

    eval_reader = CSVReader(
        input_glob=app.eval_input_fp, is_training=False, input_schema=app.input_schema, batch_size=app.eval_batch_size,
    )

    result = app.run_evaluate(reader=eval_reader, checkpoint_path=app.eval_ckpt_path)
    print(result)


In [6]:
config_json = json.load(open("user_config.json", "r", encoding="utf-8"))
train(config_json)

INFO:tensorflow:***************** modelZooBasePath ./eztransfer_modelzoo\.eztransfer_modelzoo ***************
INFO:tensorflow:total number of training examples 1000
INFO:tensorflow:***********Running in train_and_evaluate_on_the_fly mode***********
INFO:tensorflow:***********Disable Tao***********
INFO:tensorflow:***********NCCL_IB_DISABLE 0***********
INFO:tensorflow:***********NCCL_P2P_DISABLE 0***********
INFO:tensorflow:***********NCCL_SHM_DISABLE 0***********
INFO:tensorflow:***********NCCL_MAX_NRINGS 4***********
INFO:tensorflow:***********NCCL_MIN_NRINGS 2***********
INFO:tensorflow:***********NCCL_LAUNCH_MODE PARALLEL***********
INFO:tensorflow:***********TF_JIT_PROFILING False***********
INFO:tensorflow:***********PAI_ENABLE_HLO_DUMPER False***********
INFO:tensorflow:***********Single worker, Single gpu, Don't use distribution strategy***********
INFO:tensorflow:model_dir: model_dir
INFO:tensorflow:num workers: 1
INFO:tensorflow:num gpus: 1
INFO:tensorflow:learning rate: 1e-0

C:\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
C:\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 31: model_dir\model.ckpt-31
INFO:tensorflow:Saving checkpoints for 62 into model_dir\model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (100 secs).
INFO:tensorflow:Saving checkpoints for 93 into model_dir\model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (100 secs).
INFO:tensorflow:global_step/sec: 1.37586
INFO:tensorflow:loss = 2.14808, step = 100 (72.684 sec)
INFO:tensorflow:Saving checkpoints for 124 into model_dir\model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (100 secs).
INFO:tensorflow:Saving checkpoints for 155 into model_dir\model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (100 secs).
INFO:tensorflow:Saving checkpoints for 157 into model_dir\model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (100 secs).
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Lo

C:\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
C:\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


INFO:tensorflow:Loss for final step: 1.8996964.


In [7]:
config_json["evaluate_config"]["eval_checkpoint_path"] = config_json["evaluate_config"]["eval_checkpoint_path"].format(157)
evaluate(config_json)
# {'loss': 2.1276107, 'py_accuracy': 0.41, 'py_macro_f1': 0.37596843, 'py_micro_f1': 0.41, 'py_weighted_f1': 0.40845755, 'global_step': 157}

INFO:tensorflow:***************** modelZooBasePath ./eztransfer_modelzoo\.eztransfer_modelzoo ***************
INFO:tensorflow:num eval steps: None
INFO:tensorflow:***********Running in evaluate_on_the_fly mode***********
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\tzh\\AppData\\Local\\Temp\\tmpcohh14xs', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': intra_op_parallelism_threads: 1024
inter_op_parallelism_threads: 1024
gpu_options {
  per_process_gpu_memory_fraction: 1.0
  allow_growth: true
  force_gpu_compatible: true
}
allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001CFC72A2D88>, 

C:\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
C:\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 157: model_dir/model.ckpt-157
{'loss': 2.1276407, 'py_accuracy': 0.41, 'py_macro_f1': 0.37596843, 'py_micro_f1': 0.41, 'py_weighted_f1': 0.40845755, 'global_step': 157}
